# <p style='text-align: center;'>INFORME LABORATORIO 3: COMPUTACIÓN CUÁNTICA EN LA BIOINFORMÁTICA</p>
<p style='text-align: center;'>MANUEL ALEJANDRO QUISTIAL JURADO</p>
<p style='text-align: center;'>EDUARDO JOSÉ MAYA RODRIGUEZ</p>

## INTRODUCCIÓN
<p style='text-align: justify;'>
    La Bioinformática es una subdisciplina de la biología y las ciencias computacionales encargada de adquirir, almacenar, analizar y diseminar la información biológica, en gran parte correspondiente a las secuencias de ADN y aminoácidos. La Bioinformática se dedica al uso de programas informáticos, multiples ejemplos del campo de acción de la bioinformática son: determinar las funciones de genes y proteínas, establecer relaciones evolutivas y predecir la conformación tridimensional de las proteínas.</p>

<p style='text-align: justify;'>
    Actualmente la computación, como en este caso la computación cuántica buscan que el desarrollo de estos campos de estudio puedan ser repotenciados para lograr nuevas metas como los mencionados anteriormente ya que con la capacidad de calculo de los procesadores cuanticos se lleven a cabo desarrollos que por limitantes tecnologicos han quedado atras.</p>
    
<p style='text-align: justify;'>
    Para efectos de este laboratorio, en el cual se busca efectuar comparación de cadenas de secuencias geneticas, buscan de una manera mas sencilla implementar sistemas enfocados al cálculo y simulación de entornos biológicos más complejos como la cadena proteínica de algunas especies de animales, reforzar la ingeniería genética y analísis químicos entre otros.</p>

In [1]:
import sys
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.extensions import Initialize
%matplotlib inline

## DESARROLLO DE LA PRÁCTICA
### Análisis de un ejemplo del Tutorial del Qiskit

In [2]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    create a circuit for constructing the quantum superposition of the bitstring
    """
    n = math.ceil(math.log2(len(bitstring))) + 1 #number of qubits
    assert n > 2, "the length of bitstring must be at least 2"
    
    qc = QuantumCircuit(qr, cr)
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ]) #initialize to zero
    
############################################################################################################
    # Cambio requerido en el código para corregir error presentado #
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización
############################################################################################################
    amplitude = np.sqrt(1.0/2**(n-1))
    for i, b in enumerate(bitstring):
        pos = i * 2
        if b == "1" or b == "M":
            pos += 1
        desired_vector[pos] = amplitude
############################################################################################################
    # Cambio requerido en el código para corregir error presentado #
    qc_init = Initialize(desired_vector)
    
############################################################################################################
    if not inverse:
        # qc.initialize(desired_vector, qr)
        qc.append(qc_init, qr) # Cambio requerido en el código para corregir error presentado #
        qc.barrier(qr)
    else:
    # qc.initialize(desired_vector, qr).inverse() #invert the circuit
        inverse_qc_init = qc_init.gates_to_uncompute() # Call to create a circuit with gates that take the desired vector to zero
        qc.append(inverse_qc_init, qr) # Cambio requerido en el código para corregir error presentado #
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    return qc

In [3]:
YEAST     = "----------------------------------MM----------------------------"
PROTOZOAN = "--MM---------------M------------MMMM---------------M------------"
BACTERIAL = "---M---------------M------------MMMM---------------M------------"


In [4]:
n = math.ceil(math.log2(len(YEAST))) + 1                 #number of qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_yeast     = encode_bitstring(YEAST, qr, cr)
qc_protozoan = encode_bitstring(PROTOZOAN, qr, cr)
qc_bacterial = encode_bitstring(BACTERIAL, qr, cr)

circs = {"YEAST": qc_yeast, "PROTOZOAN": qc_protozoan, "BACTERIAL": qc_bacterial}

In [5]:
inverse_qc_yeast     = encode_bitstring(YEAST,     qr, cr, inverse=True)
inverse_qc_protozoan = encode_bitstring(PROTOZOAN, qr, cr, inverse=True)
inverse_qc_bacterial = encode_bitstring(BACTERIAL, qr, cr, inverse=True)

inverse_circs = {"YEAST": inverse_qc_yeast, "PROTOZOAN": inverse_qc_protozoan, "BACTERIAL": inverse_qc_bacterial}

In [6]:
key = "PROTOZOAN"       #the name of the code used as key to find similar ones
#key = "YEAST"
# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 1000

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key] + inverse_circs[other_key]   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of PROTOZOAN and YEAST is 0.819
Similarity score of PROTOZOAN and BACTERIAL is 0.971
[ANSWER] PROTOZOAN is most similar to BACTERIAL


<ul>
    <li>Explique el modo utilizado en este ejemplo para representar cuánticamente las tres secuencias de strings asociadas a los códigos genéticos del Yeast, Protozoan y Bacterial.</li>
    <p style='text-align: justify;'>
        Se emplea un vector que esta dado por una amplitud de 1/8 debido a que es base 2 la codificacion, 1 y 0; teniendo en cuenta la longitud del bitstring de 64,
        <br>A partir de ello se forman los estados cuanticos, donde se anexa un bit de control en 1 que determina si esta o no el codigo genetico 'M'.
        <br>Al estar lista la representacion binaria con el bit de control, se inicializa el estado cuantico.
    </p>
</ul>  
<ul>
    <li>En sus propias palabras explique la función encode_bitstring( ).</li>
    <p style='text-align: justify;'>
        Esta funcion convierte cada caracter del codigo genetico en una cadena de bits determinado por su posicion y si se encuentra o no habilitado el gen, permitiendo a partir de la cadena de bits de cada gen convertirlo en un estado cuantico y a la vez generando el circuito cuantico y el circutio cuantico inverso.
    </p>
</ul>    
<ul>
    <li>Explique la necesidad de invertir cada uno de los tres circuitos cuánticos creados para las tres
secuencias de strings que son comparadas en este ejemplo.</li>
    <p style='text-align: justify;'>
       La inversa se aplica para encontrar el estado inicial que tiene mayor similitud al estado inicial empleado en los estados cuanticos del proceso.
    </p>
</ul>   
<ul>
    <li>Explique la manera como en este ejemplo se realiza la comparación entre las secuencias de
strings dadas.</li>
    <p style='text-align: justify;'>
        Al obtener el estado inicial aplicando la inversa, se obtiene el estado cuantico que mayor se asemeja con la cadena comparada, donde los puntos de mayor indice es donde el gen tieme mayor similitud, dado por la cantidad de shots en donde se presento la mayor similitud, por este motivo se escala para encontrar la incidencia de un gen en ambas secuencias.
    </p>  
</ul>

### Comparación de secuencias genéticas de diversos animales

In [7]:
#Alpha ii
CABRA = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGTCAAGGCCGCCTGGGGCAAGGTTGGCAGCAACGCTGGAGCTTATGGCGCAGAGGCTCTGGAGAGGATGTTCCTGAGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCGGCCCAGGTCAAGGGCCACGGCGAGAAGGTGGCCGCCGCGCTGACCAAAGCGGTGGGCCACCTGGACGACCTGCCCGGTACTCTGTCTGATCTGAGTGACCTGCACGCCCACAAGCTGCGTGTGGACCCGGTCAACTTTAAGCTTCTGAGCCACTCCCTGCTGGTGACCCTGGCCTGCCACCACCCCAGTGATTTCACCCCCGCGGTCCACGCCTCCCTGGACAAGTTCTTGGCCAACGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
#Alpha 2 
CABALLO = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTATGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCCAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTCCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
#Alpha A
PATO = "ATGGTGCTGTCTGCGGCTGACAAGACCAACGTCAAGGGTGTCTTCTCCAAAATCGGTGGCCATGCTGAGGAGTATGGCGCCGAGACCCTGGAGAGGATGTTCATCGCCTACCCCCAGACCAAGACCTACTTCCCCCACTTTGACCTGCAGCACGGCTCTGCTCAGATCAAGGCCCATGGCAAGAAGGTGGCGGCTGCCCTAGTTGAAGCTGTCAACCACATCGATGACATTGCGGGTGCTCTCTCCAAGCTCAGTGACCTCCACGCCCAAAAGCTCCGTGTGGACCCTGTCAACTTCAAATTCCTGGGCCACTGCTTCCTGGTGGTGGTTGCCATCCACCACCCCGCTGCCCTGACCCCAGAGGTCCACGCTTCCCTGGACAAGTTCATGTGCGCCGTGGGTGCTGTGCTGACTGCCAAGTACCGTTAG"

In [8]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    create a circuit for constructing the quantum superposition of the bitstring
    """
    n = math.ceil(math.log2(len(bitstring))) + 2                 #ahora deben ser dos qubits adicionales 
    assert n > 2, "the length of bitstring must be at least 2"
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range((2**n)) ])     #initialize to zero
    amplitude = np.sqrt(1.0/(2**(n-2)-83)) #La longitud de la cadena de proteinas no es de base 2, entonces se escala con -83

    for i, b in enumerate(bitstring):
        pos = i * 4
        if  b == "T":
            pos += 1
        elif b == "G":
            pos += 2
        elif b == "C":
            pos += 3
        desired_vector[pos] = amplitude
        
############################################################################################################
    # Cambio requerido en el código para corregir error presentado #
    qc_init = Initialize(desired_vector)
############################################################################################################
    if not inverse:
        # qc.initialize(desired_vector, qr)
        qc.append(qc_init, qr) # Cambio requerido en el código para corregir error presentado #
        qc.barrier(qr)
    else:
    # qc.initialize(desired_vector, qr).inverse() #invert the circuit
        inverse_qc_init = qc_init.gates_to_uncompute() # Cambio requerido en el código para corregir error presentado #
        qc.append(inverse_qc_init, qr) # Cambio requerido en el código para corregir error presentado #
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    return qc

In [9]:
n = math.ceil(math.log2(len(CABRA))) + 2                #number of qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_cabra    = encode_bitstring(CABRA, qr, cr)
qc_caballo = encode_bitstring(CABALLO, qr, cr)
qc_pato = encode_bitstring(PATO, qr, cr)

circs = {"CABRA": qc_cabra, "CABALLO": qc_caballo, "PATO": qc_pato}

In [10]:
inverse_qc_cabra   = encode_bitstring(CABRA,   qr, cr, inverse=True)
inverse_qc_caballo = encode_bitstring(CABALLO, qr, cr, inverse=True)
inverse_qc_pato    = encode_bitstring(PATO,    qr,  cr, inverse=True)

inverse_circs = {"CABRA": inverse_qc_cabra, "CABALLO": inverse_qc_caballo, "PATO": inverse_qc_pato}

In [11]:
key = "CABRA"       #the name of the code used as key to find similar ones
# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 1000

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key] + inverse_circs[other_key]   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of CABRA and CABALLO is 0.744
Similarity score of CABRA and PATO is 0.567
[ANSWER] CABRA is most similar to CABALLO


In [12]:
key = "CABALLO"       #the name of the code used as key to find similar ones
# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 1000

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key] + inverse_circs[other_key]   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of CABALLO and CABRA is 0.779
Similarity score of CABALLO and PATO is 0.54
[ANSWER] CABALLO is most similar to CABRA


In [13]:
key = "PATO"       #the name of the code used as key to find similar ones
# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 1000

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key] + inverse_circs[other_key]   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of PATO and CABRA is 0.541
Similarity score of PATO and CABALLO is 0.555
[ANSWER] PATO is most similar to CABALLO


<ul>
    <li>Cómo puede representar estos cuatro nucleótidos, en su nuevo programa descrito en
Qiskit?</li>
    <p style='text-align: justify;'>
        Las cuatro bases nitrogenadas al ser expresadas en base 2, tiene una combinatoria de 2, desde 00 a 11 para cada base, de esta manera se anexa este habilitador a cada una de las n combinatorias determinada por la longitud de la cadena de genes, siendo esta la parte clave para que el algoritmo base del gen expresado 'M', pueda funcionar con las cuatro bases nitrogenadas.
    </p>
</ul>  
<ul>
    <li>Cómo se puede representar en Qiskit cada una de las secuencias genéticas indicadas en
la página anterior?</li>
    <p style='text-align: justify;'>
        Teniendo presente la respuesta a la anterior pregunta y la explicacion base de la comparacion del gen 'M', cada base nitrogenada se encuentra en una posicion que se determina por el habilitador, ya sea 00 a 11, a partir de esto se diseña el vector del estado cuantico, con la consideracion de que se trabaja en base 2
    </p>
</ul>  

## CONCLUSIONES

<ul>
    <li><p style='text-align: justify;'>
        Con este ejemplo base se abre la puerta a comprender la teoria y las ventajas que tiene la computacion cuantica, poder analizar bases nitrogenadas mas grandes para analizar el comportamiento de los virus, bacterias en ciertas bases nitrogenadas del ser humano y sus posibles consecuencias.</p></li>
    <li><p style='text-align: justify;'>
        La inversa desde la base matematica permite "retroceder" en un proceso y obtener las condiciones iniciales, de esta manera al aplicarlo al resultado del circuito cuantico, se generan las combinaciones que mayor similitud, donde re reconstruye a partir de la comparacion de las secuencias los genes con mayor prevalencia.</p></li>
    <li><p style='text-align: justify;'>
        Se puede ver en terminos de complejidad computacional como son efectuadas la simulación de las cadenas proteínicas vistos en este informe, que, en terminos de la notación de Big O que su conumo de recursos no es mas que de una forma logaritmica $O(\log(N))$, lo que resume de manera abismal su desempeño frente a sitemascomputacionales clásicos. puesto que en la medida que aumenta la cantidad de caracteres en la cadena su comportamiento en terminos de tiempo de respuesta puede ser menor en una máquina cuántica que en una máquina clásica</p></li>
    <li><p style='text-align: justify;'>
        El campo de la bioinformática y la ingeniería genética podría pensar a futuro, con la mejora y desarrollo de las máquinas cuánticas, mayores condiciones para simular diferentes problemas que en la actualidad son puestos en espera debido a las limitaciones físicas de equipos. Facilitando la mejora continua de fabricación de cultivos mas resistentes y productivos, resolución de curas para enfermedades y mejora en la calidad de vida del ser humano. 
        </p></li>
 
</ul>  

## REFERENCIAS BIBLIOGRÁFICAS

1. [Inverse gate](https://cirq.readthedocs.io/en/stable/gates.htm)
2. [Math Inverse ](https://math.stackexchange.com/questions/1684029/why-is-being-onto-necessary-for-a-function-to-have-inverse)
3. [Gate to uncompute](https://qiskit.org/documentation/stubs/qiskit.extensions.Initialize.gates_to_uncompute.html?highlight=gates_to_uncompute)
4. [String comparasion](https://github.com/Qiskit/qiskit-community-tutorials/blob/master/hello_world/string_comparison.ipynb)
5. [Bioinformática](https://www.genome.gov/es/genetics-glossary/Bioinformatica)
6. [Strings, Languajes and DFA's](https://courses.engr.illinois.edu/cs373/sp2009/lectures/lect_02.pdf)